![link text](https://i.imgur.com/v5VlmcB.png)
* Neste documento será apresentado uma Rede Neural que irá prever um número em uma imagem usando o dataset MNIST
* Os valores definidos para algumas variáveis podem não ser os melhores, porém o objetivo é aprender o funcionamento básico e não o obter um ótimo desempenho

            






##Requesitos:
* Conceitos básicos:
  * listas
  * matrizes
  * lista de matrizes

* Instalar nesta ordem:
  * Pip3 -> sudo apt-get -y install python3-pip
  * NumPy -> sudo pip3 install numpy
  * TensorFlow -> sudo pip3 install tensorflow
  * Keras -> sudo pip3 install keras
  * OpenCV -> sudo pip3 install opencv-python

##A primeira coisa para fazer é a importação das bibliotecas utilizadas

In [0]:
import cv2                               # OpenCV
import numpy as np                       # NumPy

from keras.datasets import mnist         # Importando o dataset usado no treino
from keras.models import Sequential      # Modelo de rede neural
from keras.layers import Dense           # Layer do tipo densamente conectado
from keras.utils import np_utils         # Usaremos dela o metodo 'to_categorical()'

##Carregar o Dataset MNIST
* Este dataset possui imagens em escala de cinza, vulgo preto e branco, com dimensão 28x28
*  Assim temos 60.000 imagens para treino (**x_train**) e outras 10.000 para testar (**x_test**) a performance do treino
* Além de possuir imagens, ele também possui  **y_train** e **y_test** que são uma lista de valores de 0-9 indicando a resposta para rede neural de cada imagem
* Caso a primeira imagem  de **x_train** seja o número 5, a lista **y_train** terá como seu primeiro valor o número do tipo inteiro 5

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

## Redimensionando as imagens do dataset
* As imagens do dataset estão em forma de lista de matrizes
* A variável **x_train** é uma lista com 60.000 matrizes
* Cada matriz representa a imagem de um número 
* Como as imagens possuem dimensão 28x28, as matrizes tem dimensão igual.
* Os valores que estão dentro das matrizes são os pixels de uma imagem
* Estes valores vão de 0-255, onde 0 = branco e 255 = preto

## Aplicando o método reshape()
* A dimensão de **x_train** é (60000, 28, 28), ou seja, uma lista de matrizes
* Não podemos usá-lo desta maneira na nossa rede neural, pois ela requisita uma matriz e não uma lista de matrizes
* A matriz que será criada terá uma imagem em cada linha
* Como as imagens tem resolução de 28x28, esta matriz terá 784 colunas
* Portanto a matriz possuirá 60.000 linhas e 784 colunas, uma baita matriz!
* Então, para transformar esta lista de matrizes em uma matriz usamos o método **reshape()**
* Este método será aplicado tanto em **x_train** quanto em **x_test**


In [0]:
x_train = x_train.reshape(60000, 784)             # reshape(linhas, colunas)
x_test = x_test.reshape(10000, 784)

## Aplicando normalização
* Para o melhor funcionamento será feito a normalização dos pixels das imagens, mudando o intervalo dos valores dos pixels
* A matriz precisa ter valores no intervalo de 0-1 cujos valores são float e não inteiros como antes
* Como estas imagens são de 8 bits, o maior valor encontrado na matriz é 255
* Então aquele pixel que tem o valor de 255 valerá agora 1.0, o pixel de valor 0 valerá 0.0, o valor 128 valerá 0.5 e assim por diante

In [0]:
x_train = x_train/255.0    
x_test = x_test/255.0

## Transformando listas em matrizes binárias
* As variáveis de respostas **y_train** e **y_test** são listas, logo não podemos utiliza-lás desta maneira na rede devido a função de perda que será utilizada logo mais na etapa de compilação, sendo preciso transformá-las em matrizes binárias
* Uma matriz binária é constituida de zeros e uns, valores do tipo inteiro
* A quantidade de colunas desta matriz vai depender da quantidade de valores diferentes do dataset
* No nosso caso, temos imagens com valores de 0-9 no dataset (0,1,2,3,4,5,6,7,8,9), ou seja, a matriz terá 10 colunas, uma para cada número
* O número de linhas da matriz que será criada será igual ao tamanho da lista
* A lista **y_train** possui 60.000 valores inteiros e depois de feita a alteração, nossa matriz terá também 60.000 linhas
* Na matriz binária o valor 1 aparecerá apenas uma vez por linha, o restante dos valores será zero
* Para fazer esse esquema de conversão usaremos o método **to_categorical()** da **np_utils**

In [0]:
y_train = np_utils.to_categorical(y_train)        # Transformando lista em matriz
y_test =np_utils.to_categorical(y_test)

## Estrutura da Rede Neural
* O modelo escolhido é o **Sequential** por ser mais simples de entender
* Como o nome sugere, ele é usando para que a rede seja feita em camadas como a imagem abaixo
![link text](https://www.samyzaf.com/ML/pima/nn6.png)
* **Input Layer** são as estradas, no nosso caso são 784 entradas
* **Hidden Layer 1, 2** e **3** são camadas densamente conectadas, cada uma se conecta em todas as outras. Nossa rede terá apenas uma camada destas
* **Output Layer** é a saida, a resposta da rede, no nosso caso são 10 saídas, uma para cada número

## Criando a estrutura da Rede Neural
* Redes Neurais necessitam de uma **função de ativação** para terem um desempenho maior. 
* Elas são capazes de aplicarem modificações nos neuronios da nossa rede, dando importância maior para alguns e menor para outros
* Na camada densa utilizaremos a ativação do tipo **relu** por ela ser uma função mais simples o que acarreta num treinamento mais rápido
* Na camada de saída usamos a ativação do tipo **softmax** por ela ser bastante usada na classificação de classes como é o caso deste tutorial. Ela entrega como resultado uma probabilidade onde o valor mais alto é o resultado da rede

In [0]:
model = Sequential() 

# Camada Oculta
model.add(Dense(256, input_dim=784, activation='relu'))       # Adicionando a camada densa. Dense(qtde_de_neurônios, input_dim = qtde_de_entradas, Activation='tipo_de_ativação')
# 784 = qtde_pixel
# 512 = valor arbitrário, altere por valores de potência de 2 (2, 4, 8, 16, 32, 64...)

# Camada de Saída
model.add(Dense(10, activation='softmax'))                    # Adicionando a camada de saída Dense(qtde_de_neuronios, activation='tipo_de_ativação')
# 10 = qtde_classes


## Compilando o modelo
* Vamos configurar o processo de aprendizagem
    * Otimizador: Para cada tipo de problema existe um otimizador que obtem melhores resultados. Para o nosso caso usaremos o **sgd** (Stochastic gradient descent)
    * Função de Perda: Dependendo da saída que queremos na rede neural iremos alterá-lo. Para o nosso caso usaremos o **categorical_crossentropy**
    * Lista de métricas: Usada para mostrar o desempenho do nosso modelo. Usaremos **acc** (accuracy)

In [0]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['acc'])

## Treinando a Rede Neural
* Passamos os dados de entrada, as imagens para a rede, **x_train**, passamos também a resposta destes dados **y_train** e também a quatidade de épocas que se deseja treinar sua rede
* Alterando a quantidades de épocas pode-se aumentar a precisão da rede, mas se o valor for muito grande poderá causar **overfitting**

In [0]:
model.fit(x_train, y_train, epochs=7)

Epoch 1/7
60000/60000 [==============================] - 6s 95us/step - loss: 0.6243 - acc: 0.8470
Epoch 2/7
60000/60000 [==============================] - 5s 84us/step - loss: 0.3280 - acc: 0.9093
Epoch 3/7
60000/60000 [==============================] - 5s 81us/step - loss: 0.2788 - acc: 0.9220
Epoch 4/7
60000/60000 [==============================] - 5s 78us/step - loss: 0.2481 - acc: 0.9314
Epoch 5/7
60000/60000 [==============================] - 5s 76us/step - loss: 0.2251 - acc: 0.9381
Epoch 6/7
60000/60000 [==============================] - 5s 78us/step - loss: 0.2064 - acc: 0.9424
Epoch 7/7
60000/60000 [==============================] - 5s 81us/step - loss: 0.1911 - acc: 0.9474


## Comprovando a qualidade do treino
* Se o valor de accuracy for muito menor do que o valor gerado pelo treino significa que você causou overfiting na sua rede, ou seja, treinou-lá demais

In [0]:
evaluate = model.evaluate(x_test, y_test)

print('\nloss:{:3.2f}, accuracy:{:2.2f}'.format(evaluate[0], evaluate[1]))

10000/10000 [==============================] - 0s 36us/step

loss:0.18, accuracy:0.95


## Prevendo o valor de uma imagem
* Chegamos na parte que pra mim é a **cereja do bolo**.
* Agora veremos se a rede está reconhecendo o valor de uma imagem que iremos passar
* Podemos testa-lá utilizando uma imagem do dataset de testes **x_test** ou de uma imagem que temos no computador, que você possa desenhar num editor de imagem qualquer


## Caso queira utilizar uma imagem do  computador
* Estamos usando o OpenCV para trabalhar com imagem, ele reconhece vários tipos de extensões como **jpg** e **png**
* Utilizamos ele para ler a imagem, que voce criará, **numero.jpg** do diretorio com **imread**
* Redimensionamos a nossa imagem pra ela ficar do tamanho certo usando **resize**
* Passamos ela de colorida(**RGB**) para escala de cinza(**GRAY**) com **cvtColor**, isso é necessário porque as imagens do dataset estão em escala de cinza
* A imagem deve ficar no mesmo diretorio deste algoritmo python, assim no método **imread** basta colocar o nome da foto
* Depois vamos transforma-lá em uma matriz de **1** linha por **784** colunas, porque a função de predição requesita, usando o método **reshape**
* Finalmente aplicamos a normalização

In [0]:
img = cv2.imread('numero.png')
img = cv2.resize(img, (28, 28))
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = img.reshape(1, 28*28)
img = img/255.0

error: ignored

## Caso queira utilizar uma imagem do dataset de teste
* **x_test[0]** é a primeira imagem do dataset de testes
* Precisamos transformar a imagem em uma matriz de **1** linha por **784** colunas 
* Para isso usamos o método **reshape**, 
* Não precisamos normaliza-lá porque já fizemos isto lá no inicio do código


In [0]:
img = x_test[0].reshape(1, 28*28)

## Finalmente
* Descobriremos agora se a rede neural está reconhecendo números
* A variável **resultado** irá receber do método **predict** uma matriz de probabilidades
* O valor mais alto desta matriz é o resultado da rede neural
* Para descobrir este valor mais alto usamos o método **argmax** que retorna o valor do índice da matriz onde se encontra a maior probabilidade da rede
* Caso queira descobrir o valor desta probabilidade aplique o método **max** e multiplique por **100** para ficar mais compreensível

In [0]:
resultado = model.predict(img)

print('Valor previsto: ',resultado.argmax())
print('Precisão: {:4.2f}%'.format(resultado.max()* 100))

Valor previsto:  0
Precisão: 99.27%


# Por hora é isto!
* Recomendo você alterar os valores na criação da estrutura, coloque mais uma camada oculta, altere valores no método **compile** e **fit** e observe se a precisão aumentou. 
* É de extrema importancia fazer isto, pois assim você ficará mais interessado e irá adquirir um conhecimento maior sobre Redes Neurais. 

##*Referências Bibliográficas*
[Keras: The Python Deep Learning library](https://keras.io). Acessado em 23 de fevereiro de 2019